<a href="https://colab.research.google.com/github/chhendley/Chhendley-portfolio-python/blob/main/sugar_DASH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
! pip install dash   
! pip install dash-html-components                                         
! pip install dash-core-components                                     
! pip install plotly
! pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import io
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from jupyter_dash import JupyterDash
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy import stats

visualize sugar data & time series forecast year 2022

In [10]:
region_columns = ['Year', 'Month', 'bakery, related products', 'confectionery','ice cream & dairy','beverages', 'canned, bottled & frozen foods', 'multiple food','non-food uses','hotel & restaurants',
'wholesale','retail & chain stores', 'government agencies','other', 'Monthly total']

product_columns = ['Year','Month','New England','Mid-Atlantic','North Central', 'South', 'West', 'Puerto Rico', 'Montly total']	

In [11]:
#sugar dataframe cleaning helper function
def clean_data(col_name, link):
  data = pd.read_excel(str(link))
  data.drop([0,1,2,3,4,5,6], inplace=True)
  data.fillna(0)
  data.astype(int)
  data.columns = col_name
  data['DATE'] = pd.to_datetime(data[['Year', 'Month' ]].assign(DAY=1), format='%Y-%m-%d')
  data.reset_index(inplace=True)
  data.drop(['Year', 'Month'],axis=1,inplace=True)
  return data


def clean_data1(col_name, link):
  data = pd.read_excel(str(link))
  data.drop([0,1,2,3,4, 380], inplace=True)
  data.fillna(0)
  data.astype(int)
  data.columns = col_name
  data['DATE'] = pd.to_datetime(data[['Year', 'Month']].assign(DAY=1)) 
  data.set_index(data['DATE'], inplace=True)
  data.loc['Total']= data.sum()
  data.at["Total", 'DATE'] = '0'
  data.drop(['DATE', 'Year', 'Month'],axis=1,inplace=True)
  return data

In [12]:
#clean data and store in dataframe
NE_sugar_df = clean_data(region_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_sugdane.xls')
MA_sugar_df = clean_data(region_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_sugdama.xls')
NC_sugar_df = clean_data(region_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_sugdanc.xls')
S_sugar_df = clean_data(region_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_sugdaso.xls')
W_sugar_df = clean_data(region_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_sugdawe.xls')
bakery_df = clean_data1(product_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_deltyp1.xls')
confectionary_df = clean_data1(product_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_deltyp2.xls')
ice_cream_df= clean_data1(product_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_deltyp3.xls')
beverages_df = clean_data1(product_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_deltyp4.xls')
multiple_df = clean_data1(product_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_deltyp6.xls')
nonfood_df = clean_data1(product_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_deltyp7.xls')
hotels_df = clean_data1(product_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_deltyp8.xls')
wholesale_df = clean_data1(product_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_deltyp9.xls')
retail_df = clean_data1(product_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_deltyp10.xls')
government_df = clean_data1(product_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_deltyp11.xls')
other_df = clean_data1(product_columns, 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_deltyp12.xls')

canned_df = pd.read_excel('https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/EPAS/Excel/hist_deltyp5.xls')
canned_df.drop([0,1,2,3,4, 380], inplace=True)
canned_df.fillna(0)
canned_df.columns = product_columns

<ipython-input-11-d91a5024efcb>:22: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.loc['Total']= data.sum()
<ipython-input-11-d91a5024efcb>:22: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.loc['Total']= data.sum()
<ipython-input-11-d91a5024efcb>:22: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  d

In [15]:
NE_total = NE_sugar_df['Monthly total']
MA_total= MA_sugar_df['Monthly total']
NC_total =NC_sugar_df['Monthly total']
S_total = S_sugar_df['Monthly total']
W_total = W_sugar_df['Monthly total']
totals = [NE_total, MA_total, NC_total, S_total, W_total]

totals_TS_df = pd.concat(totals, axis=0)
cols = ['NE_total', 'MA_total', 'NC_total', 'S_total', 'W_total']
totals_TS_df.columns = cols

In [29]:

from dash import Dash, dcc, html, Input, Output
from plotly.express import data
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash


app = JupyterDash(__name__)

df = NE_sugar_df


app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Northeast Sugar consumption', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),
        dcc.Dropdown( id = 'dropdown',
        options = [
            {'label': 'Bakery', 'value':'bakery, related products'},
            {'label':'Beverages', 'value':'beverages' },
            {'label': 'Canned, bottles & frozen foods', 'value':'canned, bottles & frozen foods'},
            {'label': 'Confectionery', 'value':'confectionery'},
            {'label': 'Government Agencies', 'value':'government agencies'},
            {'label': 'Hotel & Restaurants', 'value': 'hotels & restaurants'},
            {'label': 'Ice cream & Dairy', 'value':'ice cream & dairy'},
            {'label': 'Multiple food', 'value':'multiple food'},
            {'label': 'Non-food uses', 'value':'non-food uses'},
            {'label': 'Other', 'value':'other'},
            {'label': 'Retail & Chain stores', 'value':'retail & chain stores'},
            {'label': 'Total', 'value':'montly total'},
            {'label': 'Wholesale', 'value':'wholesale'},
            ],
        value = 'GOOG'),
        dcc.Graph(id = 'bar_plot')
    ])
    
    
@app.callback(Output(component_id='bar_plot', component_property= 'figure'),
              Input(component_id='dropdown', component_property= 'value'))
def graph_update(dropdown_value):
    print(dropdown_value)
    fig = go.Figure([go.Scatter(x = df['DATE'], y = df['{}'.format(dropdown_value)],\
                     line = dict(color = 'firebrick', width = 4))
                     ])
    
    fig.update_layout(title = 'Sugar prices over time',
                      xaxis_title = 'Dates',
                      yaxis_title = 'Prices'
                      )
    return fig  


if __name__ == '__main__': 
    app.run_server(mode="inline")



Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



<IPython.core.display.Javascript object>